In [1]:
import libsbml
import pandas as pd
import numpy as np
from tools import *

In [2]:
model_name = "example"
xlsx_file = f"Sources/Template_Toy.xlsx"
sbml_file = f"Sources/{model_name}_boolnet.sbml"
save_file = f"Models/{model_name}.sbml"
strip_file = f"Models/{model_name}_no_annotation.sbml"

In [4]:
# Load SBML-qual model
reader = libsbml.SBMLReader()
document = reader.readSBML(sbml_file)
model = document.getModel().getPlugin('qual')

## Model statistics

In [5]:
num_species = model.getNumQualitativeSpecies()
num_transitions = model.getNumTransitions()
print(f"Number of qualitative species: {num_species}")
print(f"Number of transitions: {num_transitions}")

Number of qualitative species: 5
Number of transitions: 5


## Add metaids to all components


In [6]:
# Ensure each component in the SBML model has a metaid.
def generateMetaId(index):
    return f"metaid_{index:07d}"
    
index = 1
# Process the model
document.getModel().setMetaId(generateMetaId(index))

# Process species
for species in model.getListOfQualitativeSpecies():
    index += 1
    if not species.isSetMetaId():
        species.setMetaId(generateMetaId(index))

# Process transitions
for transition in model.getListOfTransitions():
    index += 1
    if not transition.isSetMetaId():
        transition.setMetaId(generateMetaId(index))
    
    # set metaid for inputs of transition
    for input in transition.getListOfInputs():
        index += 1
        if not input.isSetMetaId():
            input.setMetaId(generateMetaId(index))
    
    # set metaid for outputs of transition
    for output in transition.getListOfOutputs():
        index += 1
        if not output.isSetMetaId():
            output.setMetaId(generateMetaId(index))

# libsbml.writeSBMLToFile(document, 'test.sbml')

## Species annotation from spreadsheet

In [7]:
# xlsx tab with the species annotations
reference_df = pd.read_excel(xlsx_file, sheet_name='Nodes', dtype=str)
reference_df.fillna('', inplace=True)
reference_df

,Compartment,ID,Name,Type,maxLevel,Constant,Relation,Resource,Identifier,Relation2,Resource2,Identifier2,Notes,Comments
0,cytosol,A,RELA,Internal,1,false,is,uniprot,P06539,,,,p65 subunit of the NF-κB complex,Hypothetical annotation
1,cytosol,B,NFKB1,Internal,1,false,is,uniprot,P19838,,,,p50 subunit of the NF-κB complex,Hypothetical annotation
2,cytosol,C,NFκB_complex,Internal,1,false,hasPart,uniprot,P06539;P19838,,,,RELA:p50 heterodimer,Hypothetical annotation
3,cytosol,D,IKK,input,1,true,is,uniprot,O14920,,,,IKBKB,Hypothetical annotation
4,extracellular,E,Inflammatory response,output,1,false,is,go,GO:0006954,isVersionOf,go,GO:0005125,Pro-inflammatory cytokine expression,Hypothetical annotation


### Add Notes to species

In [8]:
# Go over each row in the reference_df
for index, row in reference_df.iterrows():
    # Get the species id
    species_id = row['ID']
    oneSpecies = model.getQualitativeSpecies(species_id)
    if oneSpecies is not None:
        if row['Notes'] != '':
            oneSpecies.setNotes(row['Notes'], True)

# libsbml.writeSBMLToFile(document, 'test.sbml')

### Annotate biological identity

In [9]:
"""
Add biological identifiers to species in the model based on reference DataFrame.
Assuming no existing annotation in the model (otherwise, they will be overwritten)

Parameters:
-----------
model: libsbml.Model
    The SBML model to annotate
reference_df: pandas.DataFrame
    DataFrame containing species information with Relation, Resource, Identifier columns
    May also contain Relation2, Resource2, Identifier2, etc. for multiple resources
"""

for index, row in reference_df.iterrows():
    # Get the species id
    species_id = row['ID']
    oneSpecies = model.getQualitativeSpecies(species_id)
    
    if oneSpecies is None:
        print(f"Species {species_id} not found in model")
        continue  # Skip if species not found in model
    
    # Ensure species has a metaid
    if not oneSpecies.isSetMetaId():
        meta_id = f"metaid_{index:07d}"
        oneSpecies.setMetaId(meta_id)
    else:
        meta_id = oneSpecies.getMetaId()
    
    # Dictionary to collect all identifiers by relation and resource
    all_annotations = {}
    
    # Process primary identifiers if they exist
    if pd.notna(row.get('Resource', None)) and pd.notna(row.get('Identifier', None)):
        relation = row['Relation'] if pd.notna(row['Relation']) else 'is'
        resource = row['Resource']
        
        # Initialize the dictionary entry if it doesn't exist
        if relation not in all_annotations:
            all_annotations[relation] = {}
        if resource not in all_annotations[relation]:
            all_annotations[relation][resource] = []
            
        # Add identifiers to the list
        identifiers = str(row['Identifier']).split(';')
        all_annotations[relation][resource].extend([id.strip() for id in identifiers if id.strip()])
    
    # Process additional identifiers (Relation2, Resource2, Identifier2, etc.)
    suffix = 2
    while f'Resource{suffix}' in row and pd.notna(row[f'Resource{suffix}']) and \
            f'Identifier{suffix}' in row and pd.notna(row[f'Identifier{suffix}']):
        
        relation = row[f'Relation{suffix}'] if pd.notna(row[f'Relation{suffix}']) else 'is'
        resource = row[f'Resource{suffix}']
        
        # Initialize the dictionary entry if it doesn't exist
        if relation not in all_annotations:
            all_annotations[relation] = {}
        if resource not in all_annotations[relation]:
            all_annotations[relation][resource] = []
        
        # Add identifiers to the list
        identifiers = str(row[f'Identifier{suffix}']).split(';')
        all_annotations[relation][resource].extend([id.strip() for id in identifiers if id.strip()])
        
        suffix += 1

    # Skip if all_annotations is empty or only contains empty values
    if all_annotations and not (len(all_annotations) == 1 and '' in all_annotations and 
                                len(all_annotations['']) == 1 and '' in all_annotations[''] and 
                                not all_annotations['']['']): 
        annotation_string = createAnnotationStringFromDict(all_annotations, meta_id)
        # Set the complete annotation
        oneSpecies.setAnnotation(annotation_string)
        print(f"Added annotations to {species_id}")

libsbml.writeSBMLToFile(document, 'test.sbml')

Added annotations to A
Added annotations to B
Added annotations to C
Added annotations to D
Added annotations to E


1

### Check which species have no annotation

In [10]:
unannotated_species = []
for species in model.getListOfQualitativeSpecies():
    if not species.isSetAnnotation():
        unannotated_species.append(species.getId())
print(len(unannotated_species))

0


## Transition annotation from spreadsheet


In [11]:
# xlsx tab with the species annotations
reference_df = pd.read_excel(xlsx_file, sheet_name='Transitions', dtype=str)
reference_df.fillna('', inplace=True)
reference_df

,Target,Level,Rules,Relation,Resource,Identifier,Notes,Comments
0,A,1,A | (D & !C),isDescribedBy,pubmed,9660950;20300203,RELA can be self-sustaining or activated by th...,Hypothetical annotation
1,B,1,A,isDescribedBy,pubmed,9417120,NFKB1 expression is induced by RELA.,Hypothetical annotation
2,C,1,A & B,isDescribedBy,pubmed,9450761,The NF-κB complex forms when both RELA and NFK...,Hypothetical annotation
3,D,1,D,isDescribedBy,pubmed,20300203,The IKK complex is considered an external or c...,Hypothetical annotation
4,E,1,C,isDescribedBy,pubmed,2183031,IL6 expression is induced by the active NF-κB ...,Hypothetical annotation


### Add Notes to transitions

In [12]:
# Create a dictionary mapping from QualitativeSpecies to transition Id
target_to_transition = {}
for transition in model.getListOfTransitions():
    transition_id = transition.getId()
    target_species = transition.getOutput(0).getQualitativeSpecies()
    target_to_transition[target_species] = transition_id

# Print the mapping for verification
for species, transition_id in target_to_transition.items():
    print(f"Species: {species} -> Transition: {transition_id}")

Species: A -> Transition: tr_A
Species: B -> Transition: tr_B
Species: C -> Transition: tr_C
Species: D -> Transition: tr_D
Species: E -> Transition: tr_E


In [13]:
# Go over each row in the reference_df
for index, row in reference_df.iterrows():
    # Get the species id
    transition_id = target_to_transition[row['Target']]
    oneTransition = model.getTransition(transition_id)
    if oneTransition is not None:
        if row['Notes'] != '':
            oneTransition.setNotes(row['Notes'], True)

# libsbml.writeSBMLToFile(document, 'test.sbml')

### Annotate transition annotations

In [14]:
"""
Add biological identifiers to transition in the model based on reference DataFrame.
Assuming no existing annotation in the model (otherwise, they will be overwritten)

Parameters:
-----------
model: libsbml.Model
    The SBML model to annotate
reference_df: pandas.DataFrame
    DataFrame containing transition information with Relation, Resource, Identifier columns
    May also contain Relation2, Resource2, Identifier2, etc. for multiple resources
"""

for index, row in reference_df.iterrows():
    # Get the species id
    transition_id = target_to_transition[row['Target']]
    oneTransition = model.getTransition(transition_id)
    
    if oneTransition is None:
        print(f"Transition targeting {row['Target']} not found in model")
        continue  # Skip if transition not found in model
    
    # Ensure transition has a metaid
    if not oneTransition.isSetMetaId():
        meta_id = f"metaid_{index:07d}"
        oneTransition.setMetaId(meta_id)
    else:
        meta_id = oneTransition.getMetaId()
    
    # Dictionary to collect all identifiers by relation and resource
    all_annotations = {}
    
    # Process primary identifiers if they exist
    if pd.notna(row.get('Resource', None)) and pd.notna(row.get('Identifier', None)):
        relation = row['Relation'] if pd.notna(row['Relation']) else 'isDescribedBy'
        resource = row['Resource']
        
        # Initialize the dictionary entry if it doesn't exist
        if relation not in all_annotations:
            all_annotations[relation] = {}
        if resource not in all_annotations[relation]:
            all_annotations[relation][resource] = []
            
        # Add identifiers to the list
        identifiers = str(row['Identifier']).split(';')
        all_annotations[relation][resource].extend([id.strip() for id in identifiers if id.strip()])
    
    # Process additional identifiers (Relation2, Resource2, Identifier2, etc.)
    suffix = 2
    while f'Resource{suffix}' in row and pd.notna(row[f'Resource{suffix}']) and \
            f'Identifier{suffix}' in row and pd.notna(row[f'Identifier{suffix}']):
        
        relation = row[f'Relation{suffix}'] if pd.notna(row[f'Relation{suffix}']) else 'is'
        resource = row[f'Resource{suffix}']
        
        # Initialize the dictionary entry if it doesn't exist
        if relation not in all_annotations:
            all_annotations[relation] = {}
        if resource not in all_annotations[relation]:
            all_annotations[relation][resource] = []
        
        # Add identifiers to the list
        identifiers = str(row[f'Identifier{suffix}']).split(';')
        all_annotations[relation][resource].extend([id.strip() for id in identifiers if id.strip()])
        
        suffix += 1

    # Skip if all_annotations is empty or only contains empty values
    if all_annotations and not (len(all_annotations) == 1 and '' in all_annotations and 
                                len(all_annotations['']) == 1 and '' in all_annotations[''] and 
                                not all_annotations['']['']): 
        annotation_string = createAnnotationStringFromDict(all_annotations, meta_id)
        # Set the complete annotation
        oneTransition.setAnnotation(annotation_string)
        print(f"Added annotations to {transition_id}")

libsbml.writeSBMLToFile(document, 'test.sbml')

Added annotations to tr_A
Added annotations to tr_B
Added annotations to tr_C
Added annotations to tr_D
Added annotations to tr_E


1

## Save the model

In [15]:
libsbml.writeSBMLToFile(document, save_file)

1

## Strip annotations and notes

In [24]:
model_name = "Faure2006"
save_file = f"Models/{model_name}.sbml"
strip_file = f"Models/{model_name}_no_annotation.sbml"

reader = libsbml.SBMLReader()
document = reader.readSBML(save_file)
model = document.getModel().getPlugin('qual')

# Process species
for species in model.getListOfQualitativeSpecies():
    species.unsetAnnotation()
    species.unsetNotes()

# Process transitions
for transition in model.getListOfTransitions():
    transition.unsetAnnotation()
    transition.unsetNotes()
    for input in transition.getListOfInputs():
        input.unsetAnnotation()
        input.unsetNotes()
    for output in transition.getListOfOutputs():
        output.unsetAnnotation()
        output.unsetNotes()

libsbml.writeSBMLToFile(document, strip_file)

1

## Test

In [ ]:
def merge_annotations(species, new_annotation_string, meta_id):
    """
    Properly merge a new annotation string with existing species annotations.
    
    Parameters:
    -----------
    species: libsbml.QualitativeSpecies
        The species to modify
    new_annotation_string: str
        The new annotation to add
    meta_id: str
        The meta ID of the species
        
    Returns:
    --------
    bool
        True if successful, False otherwise
    """
    import re
    
    # If there are no existing annotations, just set the new one
    if not species.isSetAnnotation():
        species.setAnnotation(new_annotation_string)
        return True
    
    current_annotation = species.getAnnotationString()
    
    # Extract just the bag content from the new annotation
    bag_pattern = re.compile(r'<bqbiol:[^>]+>\s*<rdf:Bag>(.+?)</rdf:Bag>\s*</bqbiol:[^>]+>', re.DOTALL)
    match = bag_pattern.search(new_annotation_string)
    
    if not match:
        return False
    
    bag_content = match.group(1).strip()
    qualifier_match = re.search(r'<bqbiol:([^>]+)>', new_annotation_string)
    qualifier = qualifier_match.group(1) if qualifier_match else "is"
    
    # Check if the qualifier already exists in the current annotation
    qualifier_pattern = re.compile(f'<bqbiol:{qualifier}>[^<]*<rdf:Bag>(.+?)</rdf:Bag>[^<]*</bqbiol:{qualifier}>', re.DOTALL)
    qualifier_match = qualifier_pattern.search(current_annotation)
    
    if qualifier_match:
        # Add to existing qualifier bag
        existing_bag = qualifier_match.group(1)
        new_bag = existing_bag + bag_content
        updated_annotation = current_annotation.replace(existing_bag, new_bag)
        species.setAnnotation(updated_annotation)
    else:
        # Add as a new qualifier
        insertion_point = current_annotation.find('</rdf:Description>')
        if insertion_point > 0:
            # Construct new qualifier block
            new_qualifier_block = f'<bqbiol:{qualifier}><rdf:Bag>{bag_content}</rdf:Bag></bqbiol:{qualifier}>'
            
            # Insert before closing tags
            updated_annotation = current_annotation[:insertion_point] + new_qualifier_block + current_annotation[insertion_point:]
            species.setAnnotation(updated_annotation)
        else:
            return False
    
    return True